# 0. Libraries

In [3]:
import numpy as np
import pandas as pd

# 1. Extract the data

In [6]:
# Download the articles CSV file
!wget https://raw.githubusercontent.com/jgalazka/SB_publications/main/SB_publication_PMC.csv -O SB_publication_PMC.csv

--2025-09-19 12:31:29--  https://raw.githubusercontent.com/jgalazka/SB_publications/main/SB_publication_PMC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8000::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97057 (95K) [text/plain]
Saving to: ‘SB_publication_PMC.csv’

SB_publication_PMC. 100%[===================>]  94.78K  --.-KB/s    in 0.07s   

2025-09-19 12:31:30 (1.40 MB/s) - ‘SB_publication_PMC.csv’ saved [97057/97057]



In [5]:
df = pd.read_csv("SB_publication_PMC.csv")
df.head()

,Title,Link
0,Mice in Bion-M 1 space mission: training and s...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
1,Microgravity induces pelvic bone loss through ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...
2,Stem Cell Health and Tissue Regeneration in Mi...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,Microgravity Reduces the Differentiation and R...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
4,Microgravity validation of a novel system for ...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
